In [ ]:
# 自定义异常
class Inputerror(Exception):
    def __init__(self, messages):
        super().__init__(messages)


def sort_tuple(dist):
    # 传入字典，按照键大小顺序重排序，一个排序的过程
    return sorted(dist.items(), key=lambda x: x[1], reverse=True)


def get_coding_schedule(end1, end2, sort_list, code_schedule):
    # 传入 末端2位字符组 频数 序列列表(剔除末端字符) 哈夫曼编码表
    '''
    哈夫曼表构造过程
        传入 end1 作为右子树，end2 作为左子树
        分别判断 end1 和 end2 的字符长度，如果长度为 1 说明该字符是叶子节点，否则说明该字符是分支节点
            如果 end1 是叶子节点，则设置编码值为 "1"，如果 end2 是叶子节点，则设置编码值为 "0"
            如果 end1 是分支节点，则根据分支节点的字符串进行遍历，为每一个子叶编码值都添加前缀字符 "1"，如果 end2 是分支节点，则根据分支节点的字符串进行遍历，为每一个子叶编码值都添加前缀字符 "0"
        在 sort_list 中添加由 end1 和 end2 构成的分支节点信息，结点信息包含所有子叶字符，所有子叶累计频数
    '''
    if len(end1[0]) == 1:
        code_schedule.setdefault(end1[0], '1')
    else:
        for k in end1[0]:
            code_schedule[k] = '1' + code_schedule[k]
    if len(end2[0]) == 1:
        code_schedule.setdefault(end2[0], '0')
    else:
        for k in end2[0]:
            code_schedule[k] = '0' + code_schedule[k]
    sort_list.append((end2[0] + end1[0], end1[1] + end2[1]))
    return code_schedule


def get_keys(dict, value):
    # 传入字典，值，获取对应的键
    for k, v in dict.items():
        if v == value:
            return k


def check_binary(input_data):
    # 检查文件编码，ASCII码超出255的字符替换为空格
    output_data = ''
    for word_index in range(len(input_data)):
        if ord(input_data[word_index]) >= 256:
            output_data += ' '
        else:
            output_data += input_data[word_index]
    return output_data


def compress(file_name):
    import os
    import six
    # 1.打开文件
    f = open(file_name, 'r')

    # 2.读取信息
    file_data = check_binary(f.read())
    f.close()

    # 3.统计各字符的频数，保存在字典 char_freq 中
    char_freq = {}
    for word in file_data:
        char_freq.setdefault(word, 0)
        char_freq[word] += 1

    # 4.编码哈夫曼树
    # 4.1 初始 字符--频数 列表
    sort_list = sort_tuple(char_freq)
    # 4.2 哈夫曼编码表
    code_schedule = {}
    # 4.3 不断重排序，更新哈夫曼编码表及树节点信息
    for i in range(len(sort_list) - 1):
        # 排序
        sort_list = sort_tuple(dict(sort_list))
        # 构造树
        code_schedule = get_coding_schedule(sort_list.pop(), sort_list.pop(), sort_list, code_schedule)
        
    for j in code_schedule:
        print(j,"  :  ",code_schedule[j])
    
    # 5.文本信息转哈夫曼码
    # 5.1 夫曼 0-1 编码转码 + 正文文本
    code = ''.join(list(code_schedule.values()))
    for word in file_data:
        code += code_schedule[word]
    # 5.2 不足 8 位补 0，记录在 code_sup 中
    code_sup = 8 - len(code) % 8
    code += code_sup * '0'

    # 6.创建压缩文件
    f = open(os.path.splitext(file_name)[0] + '.qlh', 'wb')
    # 6.1 写入补 0 信息
    f.write(six.int2byte(code_sup))
    # 6.2 写入哈夫曼编码表（总长度+每一个编码长度+每一个编码对应的字符+转码信息）
    # 6.2.1 码表总长度（字符个数，与指针读取定位有关，分割码表与正文）
    f.write(six.int2byte(len(code_schedule)))
    # 6.2.1 储存每一个哈夫曼编码的位长
    for v in code_schedule.values():
        f.write(six.int2byte(len(v)))
    # 6.2.3 储存每一个哈夫曼编码配对字符        字符 ==> ASCII 码
    for k in code_schedule.keys():
        f.write(six.int2byte(ord(k)))
    # 6.3 以 8 为长度单位，将 0-1 字符转为对应的十进制数，映射为 ASCII 符号，写入正文文本
    for i in range(len(code) // 8):
        f.write(six.int2byte(int(code[8 * i:8 + 8 * i], 2)))
    # 关闭文件
    f.flush()
    f.close()
    print('压缩完成', file_name, '>>', os.path.splitext(file_name)[0] + '.qlh')


def decompress(file_name):
    import os
    # 1.打开文件
    f = open(file_name, 'rb')
    # 2.读取信息
    file_data = f.read()
    f.close()

    # 3.分割信息
    # 3.1 获取补 0 位数
    code_sup = file_data[0]
    # 3.2 获取码表长度
    code_schedule_length = file_data[1]
    # 3.3 指针跳过 补0+码长+码符
    pointer = 2 * code_schedule_length + 2
    # 3.4 获取码表中每一个编码的长度
    code_word_len = [file_data[2 + i] for i in range(code_schedule_length)]
    # 3.5 编码表中字符长度总和，用于切割码表与正文
    sum_code_word_len = sum(code_word_len) // 8 + 1 if sum(code_word_len) % 8 != 0 else sum(code_word_len) // 8

    # 4.还原码表
    # 4.1 码表转译
    code_schedule_msg = ''
    for i in range(sum_code_word_len):
        code_schedule_msg += '0' * (10 - len(bin(file_data[pointer + i]))) + bin(file_data[pointer + i])[2:]
    # 4.2 初始化指针
    pointer = 0
    # 4.3 创建码表
    code_schedule = {}
    for i in range(code_schedule_length):
        code_word = chr(file_data[code_schedule_length + 2 + i])  # 码符
        code_schedule[code_word] = code_schedule_msg[pointer:pointer + code_word_len[i]]  # 码符码文匹配，还原码表
        pointer += code_word_len[i]

    # 5.提取正文
    code = code_schedule_msg[pointer:]
    pointer = 2 * code_schedule_length + 2 + sum_code_word_len
    for number in file_data[pointer:]:
        code += '0' * (10 - len(bin(number))) + bin(number)[2:]
    # 删去补0
    code = code[:-code_sup]

    # 6.文本转译
    pointer = 0  # 指针归零
    # 初始化文本
    letter = ''
    # 限制最大搜索长度，提高效率
    max_length = max([len(list(code_schedule.values())[i]) for i in range(len(code_schedule.values()))])
    while pointer != len(code):
        for i in range(max_length):
            if code[pointer:pointer + i + 1] in code_schedule.values():
                letter += get_keys(code_schedule, code[pointer:pointer + i + 1])
                pointer += i + 1
                break

    # 7.创建解压文件
    f = open(os.path.splitext(file_name)[0] + '.txt', 'w+')
    f.write(letter)
    print('解压完成', file_name, '>>', os.path.splitext(file_name)[0] + '.txt')


def compress_all(file_names):
    # 批量压缩文件
    for file_name in file_names:
        compress(file_name)


def decompress_all(file_names):
    # 批量解压文件
    for file_name in file_names:
        decompress(file_name)


def get_request():
    file_name = tkinter.filedialog.askopenfilenames()
    ask = input('Compress or Decompress ? (C/D)').lower()
    if ask == 'd':
        decompress_all(file_name)  # 解压文件
    elif ask == 'c':
        compress_all(file_name)  # 压缩文件
    else:
        raise Inputerror("accept unknown command ,routine haven't started doing anything, please run it again")


if __name__ == '__main__':
    import tkinter.filedialog

    get_request()
    while input('Continue ? (Y/N)').lower() == 'y':
        get_request()